In [ ]:
import pandas as pd

players_df = pd.read_csv('cleaned_players.csv')
matches_df = pd.read_csv('dfb_matches.csv')

In [ ]:
# Calculate average player value per team per season (excluding missing values)
avg_team_value = (
    players_df[players_df['value'].notna()]
    .groupby(['team', 'season'])['value']
    .mean()
    .reset_index()
    .rename(columns={'value': 'avg_team_value'})
)

# Merge the average value into the main dataframe
players_with_filled_value = players_df.merge(avg_team_value, on=['team', 'season'], how='left')

# Fill missing player values with the team-season average
players_with_filled_value['value'] = players_with_filled_value['value'].fillna(players_with_filled_value['avg_team_value'])


In [ ]:
team_value_filled = (
    players_with_filled_value
    .groupby(['team', 'season'])['value']
    .sum()
    .reset_index()
    .rename(columns={'value': 'team_value'})
)


In [ ]:
# Rename for joining home and away values
home_values_filled = team_value_filled.rename(columns={
    'team': 'home_team',
    'team_value': 'home_team_value'
})
away_values_filled = team_value_filled.rename(columns={
    'team': 'away_team',
    'team_value': 'away_team_value'
})

# Merge team values into the match dataset
matches_with_filled_values = matches_df.merge(
    home_values_filled, on=['home_team', 'season'], how='left'
).merge(
    away_values_filled, on=['away_team', 'season'], how='left'
)

# Drop matches missing value info for either team
matches_with_filled_values_clean = matches_with_filled_values.dropna(subset=['home_team_value', 'away_team_value'])


In [ ]:
matches_with_filled_values_clean['value_diff'] = (
    matches_with_filled_values_clean['home_team_value'] - matches_with_filled_values_clean['away_team_value']
)

matches_with_filled_values_clean['higher_value_team'] = matches_with_filled_values_clean['value_diff'].apply(
    lambda x: 'home' if x > 0 else ('away' if x < 0 else 'equal')
)

matches_with_filled_values_clean['winner'] = matches_with_filled_values_clean.apply(
    lambda row: 'home' if row['home_score'] > row['away_score'] else (
        'away' if row['away_score'] > row['home_score'] else 'draw'
    ),
    axis=1
)


In [ ]:

import numpy as np
from scipy.stats import mannwhitneyu, ttest_ind
from statsmodels.stats.proportion import proportions_ztest
import statsmodels.api as sm

dfv = matches_with_filled_values_clean.copy()

# Ensure needed columns exist
needed_cols = ["home_division","away_division","home_score","away_score",
               "home_team_value","away_team_value","value_diff","winner"]
missing = [c for c in needed_cols if c not in dfv.columns]
if missing:
    raise ValueError(f"Missing expected columns: {missing}")

# Define 'lower_side' and 'is_upset' (drop draws, keep cross-division only)
dfv = dfv[(dfv["home_division"] != dfv["away_division"])].copy()
dfv = dfv[dfv["winner"].isin(["home","away"])].copy()  # remove draws

dfv["lower_side"] = np.where(dfv["home_division"] > dfv["away_division"], "home", "away")
dfv["is_upset"] = (dfv["winner"] == dfv["lower_side"]).astype(int)

# Use absolute value gap for comparisons
dfv["value_gap_abs"] = dfv["value_diff"].abs()

print("Counts (is_upset):\n", dfv["is_upset"].value_counts(dropna=False))

# Mann-Whitney U and (optional) t-test for |value_diff| in upsets vs non-upsets
g_upset = dfv.loc[dfv["is_upset"] == 1, "value_gap_abs"].values
g_non   = dfv.loc[dfv["is_upset"] == 0, "value_gap_abs"].values

mw_stat, mw_p = mannwhitneyu(g_upset, g_non, alternative="two-sided")
print("\nMann-Whitney U test (|value_diff| upset vs non-upset): U=%.1f, p=%.4g" % (mw_stat, mw_p))

t_stat, t_p = ttest_ind(g_upset, g_non, equal_var=False)
print("t-test (|value_diff| upset vs non-upset): t=%.2f, p=%.4g" % (t_stat, t_p))

print("Means: upset=%.3g, non-upset=%.3g | Medians: upset=%.3g, non-upset=%.3g" %
      (g_upset.mean(), g_non.mean(), np.median(g_upset), np.median(g_non)))

# Proportion test: does the higher-value team win > 50%?
df1 = matches_with_filled_values_clean.copy()
df1 = df1[df1["winner"].isin(["home","away"])].copy()
df1 = df1[(df1["home_division"] - df1["away_division"]).abs() <= 1].copy()

# Determine higher_value_team and whether that team actually won
df1["higher_value_team"] = np.where(df1["value_diff"] > 0, "home",
                                np.where(df1["value_diff"] < 0, "away", "equal"))
df1 = df1[df1["higher_value_team"] != "equal"].copy()  # drop equal-value cases

df1["higher_team_won"] = (df1["winner"] == df1["higher_value_team"]).astype(int)

succ = int(df1["higher_team_won"].sum())
nobs = int(df1["higher_team_won"].count())
stat, pval = proportions_ztest(count=succ, nobs=nobs, value=0.5)  # H0: p = 0.5
print("\nProportion test (higher-value team win rate > 0.5?)")
print("successes=%d, n=%d, p-hat=%.3f, z=%.3f, p=%.4g" % (succ, nobs, succ/nobs if nobs else float('nan'), stat, pval))


In [ ]:
# Keep only matches where teams are in the same division or ±1 division
matches_filtered_division_filled = matches_with_filled_values_clean[
    (abs(matches_with_filled_values_clean['home_division'] - matches_with_filled_values_clean['away_division']) <= 1)
]

# Check if higher value team won or not
def value_result_relation(row):
    if row['higher_value_team'] == row['winner']:
        return 'higher_team_won'
    elif row['winner'] == 'draw':
        return 'draw'
    else:
        return 'higher_team_lost'

matches_filtered_division_filled['value_outcome_relation'] = matches_filtered_division_filled.apply(value_result_relation, axis=1)

# Count final outcome results
value_outcome_counts_filled = matches_filtered_division_filled['value_outcome_relation'].value_counts().reset_index()
value_outcome_counts_filled.columns = ['Outcome', 'Count']

print(value_outcome_counts_filled)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 5))
sns.set(style="whitegrid")
sns.barplot(data=value_outcome_counts_filled, x='Outcome', y='Count', palette='viridis')
plt.title("Match Outcomes Based on Team Value (Similar Divisions)", fontsize=14)
plt.xlabel("Outcome", fontsize=12)
plt.ylabel("Number of Matches", fontsize=12)
plt.grid(True, which='major', axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import pandas as pd

from scipy.stats import mannwhitneyu, ttest_ind
from statsmodels.stats.proportion import proportions_ztest, proportion_confint


dfv = matches_with_filled_values_clean.copy()

# Keep only decided matches (no draws) and cross-division fixtures
dfv = dfv[dfv["winner"].isin(["home", "away"])].copy()
dfv = dfv[dfv["home_division"] != dfv["away_division"]].copy()

# Define which side is lower-division and whether it's an upset
dfv["lower_side"] = np.where(dfv["home_division"] > dfv["away_division"], "home", "away")
dfv["is_upset"] = (dfv["winner"] == dfv["lower_side"]).astype(int)

# Absolute value gap for distribution comparisons
dfv["value_gap_abs"] = dfv["value_diff"].abs()

print("Row counts:")
print("  Total cross-division decided matches:", len(dfv))
print("  Upsets:", int(dfv["is_upset"].sum()))
print("  Non-upsets:", int((1 - dfv["is_upset"]).sum()))

# -----------------------------
# Mann–Whitney U and Welch t-test on |value_diff|
# H0: distribution/means of |value_diff| are the same in upsets vs non-upsets
# -----------------------------
g_upset = dfv.loc[dfv["is_upset"] == 1, "value_gap_abs"].values
g_non   = dfv.loc[dfv["is_upset"] == 0, "value_gap_abs"].values

mw_stat, mw_p = mannwhitneyu(g_upset, g_non, alternative="two-sided")
t_stat, t_p   = ttest_ind(g_upset, g_non, equal_var=False)  # Welch's t-test

print("\n[Money gap distribution tests: |value_diff|]")
print("  Mann–Whitney U: U = %.2f, p = %.4g" % (mw_stat, mw_p))
print("  Welch t-test:   t = %.2f, p = %.4g" % (t_stat, t_p))
print("  Means:   upset = %.3g, non-upset = %.3g" % (g_upset.mean(), g_non.mean()))
print("  Medians: upset = %.3g, non-upset = %.3g" % (np.median(g_upset), np.median(g_non)))

# Interpretive hint for your write-up:
if mw_p < 0.05 or t_p < 0.05:
    print("  → Result: Significant difference. Upsets tend to occur with smaller money gaps.")
else:
    print("  → Result: No significant difference detected at 5% level.")


df1 = matches_with_filled_values_clean.copy()
df1 = df1[df1["winner"].isin(["home", "away"])].copy()
df1 = df1[(df1["home_division"] - df1["away_division"]).abs() <= 1].copy()

df1["higher_value_team"] = np.where(df1["value_diff"] > 0, "home",
                                np.where(df1["value_diff"] < 0, "away", "equal"))
df1 = df1[df1["higher_value_team"] != "equal"].copy()

df1["higher_team_won"] = (df1["winner"] == df1["higher_value_team"]).astype(int)

succ = int(df1["higher_team_won"].sum())
nobs = int(df1["higher_team_won"].count())
phat = (succ / nobs) if nobs else np.nan

# One-sample z-test against 0.5
z_stat, p_val = proportions_ztest(count=succ, nobs=nobs, value=0.5)

# 95% Wilson CI for the win proportion
ci_low, ci_high = proportion_confint(count=succ, nobs=nobs, alpha=0.05, method="wilson")

print("\n[Higher-value team win rate within ±1 division]")
print("  successes = %d, n = %d, p̂ = %.3f" % (succ, nobs, phat))
print("  z = %.3f, p = %.4g" % (z_stat, p_val))
print("  95%% CI (Wilson) = [%.3f, %.3f]" % (ci_low, ci_high))

if p_val < 0.05 and phat > 0.5:
    print("  → Result: Significantly above 50%% — richer teams win more often even within ±1 division.")
elif p_val < 0.05 and phat < 0.5:
    print("  → Result: Significantly below 50%% — richer teams win less often than chance (unexpected).")
else:
    print("  → Result: Not significantly different from 50%% at 5%% level.")

